# Superlattice of cylindrical holes in Si substrate

In [ ]:
%matplotlib notebook

In [ ]:
from bornagain import deg, angstrom, nm
import bornagain as ba

In [ ]:
detector_distance = 2000.0  # in mm
pixel_size = 0.17  # in mm
npx, npy = 200, 200  # number of pixels
u0, v0 = 17., -5.0  # direct beam position in mm

In [ ]:
def get_sample():
    # defining materials
    m_air = ba.HomogeneousMaterial("Air", 0.0, 0.0)
    m_substrate = ba.HomogeneousMaterial("Silicon", 5.7e-6, 1.e-7)
 
    # collection of particles
    cylinder_ff = ba.FormFactorCylinder(15*nm, 100*nm)
    cylinder = ba.Particle(m_air, cylinder_ff)
    particle_layout = ba.ParticleLayout()
    particle_layout.addParticle(cylinder, 1.0, ba.kvector_t(0.0, 0.0, -100*nm))
    substructure = ba.InterferenceFunctionFinite2DLattice.createSquare(100*nm, 0.0, 10, 10)
    interference = ba.InterferenceFunction2DSuperLattice.createSquare(2000*nm, 0.0, 30, 10)
    interference.setSubstructureIFF(substructure)
    particle_layout.setInterferenceFunction(interference)
    particle_layout.setTotalParticleSurfaceDensity(100.0/4e6)

    air_layer = ba.Layer(m_air)
    substrate_layer = ba.Layer(m_substrate)
    substrate_layer.addLayout(particle_layout)

    multi_layer = ba.MultiLayer()
    multi_layer.addLayer(air_layer)
    multi_layer.addLayer(substrate_layer)
    return multi_layer

In [ ]:
def get_rectangular_detector():
    width = npx*pixel_size
    height = npy*pixel_size
    detector = ba.RectangularDetector(npx, width, npy, height)
    detector.setPerpendicularToDirectBeam(detector_distance, u0, v0)
    return detector

In [ ]:
def get_simulation():
    simulation = ba.GISASSimulation()
    simulation.setDetector(get_rectangular_detector())
    simulation.setBeamParameters(0.13*nm, 0.18*deg, 0.0*deg)
    simulation.getOptions().setMonteCarloIntegration(True, 100)
    return simulation

In [ ]:
def run_simulation():
    simulation = get_simulation()
    simulation.setSample(get_sample())
    simulation.runSimulation()
    return simulation.result()

In [ ]:
result = run_simulation()
ba.plot_simulation_result(result)